In [49]:
import socket
import random
import time

In [50]:
def generate_random_move():
    x = random.randint(0, 3)
    y = random.randint(0, 3)
    x2 = random.randint(0, 3)
    y2 = random.randint(0, 3)
    return f"MOVE {x},{y},{x2},{y2}"

In [51]:
def connect_to_server(host='localhost', port=12345):
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client_socket.connect((host, port))
    
    response = client_socket.recv(1024).decode()
    print(f"Server ResponseINIT: {response}")
    
    Game = response[-4:]
    print("Playing:", Game)
    
    if "1" in response:
        ag=1
    else:
        ag=2
    first=True
    
    while True:
        # Generate and send a random move
        if ag == 1 or not first:
            move = generate_random_move()
            time.sleep(1)
            client_socket.send(move.encode())
            print("Send:",move)
        
            # Wait for server response
            response = client_socket.recv(1024).decode()
            print(f"Server Response1: {response}")
            if response == "INVALID":
                continue
            if "END" in response: break
         
        first=False
        response = client_socket.recv(1024).decode()
        print(f"Server Response2: {response}")
        if "END" in response: break

        # Add some condition to break the loop, if necessary
        # Example: If server sends a certain message, or after a number of moves

    client_socket.close()

In [52]:
if __name__ == "__main__":
    connect_to_server()

Server ResponseINIT: AG1 A4x4
Playing: A4x4
Send: MOVE 1,0,3,2
Server Response1: VALID
Server Response2: 
Send: MOVE 1,2,3,2


ConnectionAbortedError: [WinError 10053] Uma ligação estabelecida foi anulada pelo software no computador anfitrião